![QMUL](Images/QMUL-logo.jpg)

# Introduction to Approximate Bayesian Computation in population genetics

## Bayesian concepts

In Bayesian statistics, $\theta$ is not a fixed (although unknown) parameter
but a random quantity.

This is done by adopting a probability distribution, called _prior distribution_ , for ${\theta}$ that contains any information we have about ${\theta}$ not related to the data ${y}$.

Inferences on ${\theta}$ are based on its _posterior distribution_ given by
\begin{equation}
    P({\theta}|{y}) = \frac{f({y}|{\theta})\pi({\theta})}{m({y})} 
                        = \frac{f({y}|{\theta})\pi({\theta})}{\int f({y}|{\theta}) \pi({\theta}) d{\theta}}
\end{equation}

This formula is known as _Bayes' Theorem_.

## Normal/Normal model

Let's assume that we monitored a certain number of frogs in a given pond and want to make some inferences on the infection rate, $\theta$, in the whole area.

<img src="Images/Muletensis.jpeg" width="500" height="500" />

If both the prior and the likelihood are Normal (Gaussian) distributions, then 
\begin{align}
 f(y|\theta) &= N(y|\theta, \sigma^2)\\
 \pi(\theta) &= N(\theta|\mu, \tau^2)
\end{align}

$\mu$ and $\tau$ are known _hyperparameters_ while $\theta$ is the unknown parameter.

The posterior distribution $p(\theta|y)$ is also a Normal distribution
\begin{equation}
  p(\theta|y) = N(\theta|\frac{\sigma^2\mu+\tau^2y}{\sigma^2+\tau^2},\frac{\sigma^2\tau^2}{\sigma^2 + \tau^2})
\end{equation}

If 
\begin{equation}
  B = \frac{\sigma^2}{\sigma^2+\tau^2}
\end{equation}
then 
\begin{align}
 E(\theta|y) &= B\mu + (1-B)y \\
 Var(\theta|y) &= (1-B)\sigma^2 \equiv B\tau^2
\end{align}

$B$ is called the _shrinking factor_. 

It is called like that because it gives the proportion for how much the posterior mean is "shrunk back" from the classical frequentist estimate $y$ towards the prior mean $\mu$.

Note that $0 \leq B \leq 1$.

The posterior mean is a weighted average of the prior mean $\mu$ and the direct estimate $Y$.
The weight on the prior mean $B$ depends on the relative variability of the prior distribution and the likelihood.

If $\sigma^2>>\tau^2$

then $B \approx 1$ and our prior knowledge is more precise than the data information.

If $\sigma^2<<\tau^2$

then $B \approx 0$ and our prior knowledge is imprecise and the final estimate will move very little towards
the prior mean.

<img src="Images/Muletensis.jpeg" width="300" height="300" />

* We have a single observation from one pond of $6$ infected frogs ($y=6$).

* Our likelihood function (Normal distribution) has $\sigma=1$.

* We expected $2$ infected frogs before doing the monitoring (with variance equal to 1).

More formally,
\begin{align}
 f(y=6|\theta) &= N(y=6|\theta, 1) \\
 \pi(\theta) &= N(\theta|2, 1)
\end{align}

In [ ]:
# prior
mu <- 2
tau <- 1

x <- seq(-4,10,0.01)
plot(x=x, dnorm(x=x, mean=mu, sd=tau), ylim=c(0,0.6),
    type="l", lty=1, ylab="Density", xlab=expression(theta), main="")
    legend(x="topleft", legend=c(expression(pi(theta)),
    expression(f(y~"|"~theta)), expression(p(theta~"|"~y))), lty=1:3)

In [ ]:
plot(x=x, dnorm(x=x, mean=mu, sd=tau), ylim=c(0,0.6),
    type="l", lty=1, ylab="Density", xlab=expression(theta), main="")
    legend(x="topleft", legend=c(expression(pi(theta)),
    expression(f(y~"|"~theta)), expression(p(theta~"|"~y))), lty=1:3) # prior

# likelihood
y <- 6
sigma <- 1
points(x=x, y=dnorm(x=y, mean=x, sd=sigma), type="l", lty=2)

In [ ]:
# prior
plot(x=x, dnorm(x=x, mean=mu, sd=tau), ylim=c(0,0.6),
    type="l", lty=1, ylab="Density", xlab=expression(theta), main="")
    legend(x="topleft", legend=c(expression(pi(theta)),
    expression(f(y~"|"~theta)), expression(p(theta~"|"~y))), lty=1:3) # prior

# likelihood
points(x=x, y=dnorm(x=y, mean=x, sd=sigma), type="l", lty=2) # likelihood

# posterior
B <- sigma^2/(sigma^2+tau^2)
postMean <- B*mu + (1-B)*y
postVar <- B*tau^2
points(x=x, y=dnorm(x=x, mean=postMean, sd=sqrt(postVar)), type="l", lty=3)

The prior distribution is centered around 2 ($\mu$).
The likelihood function is centered around $6$ which is the only observation we have.

The posterior distribution is centred exactly between the prior and the likelihood.
In this case $B=0.5$ and therefore prior and data are equally weighted.

The _maximum a posteriori probability_ (MAP) estimate is $4$, as it is equal to the mode of the posterior distribution.

In [ ]:
x[which.max(dnorm(x=x, mean=postMean, sd=sqrt(postVar)))]

Suppose we have $\mu=2$, $\sigma=\tau=1$ and set $\bar{y}=5.8$ and $n=5$. In this case $\theta_{MAP}=4.43$ with a range of $(3.21, 5.65)$. The MAP has been shifted towards the MLE as we have more data information.

In [ ]:
# prior (obviously it does not change)
mu <- 2
tau <- 1
x <- seq(-4,10,0.01)
plot(x=x, dnorm(x=x, mean=mu, sd=tau), ylim=c(0,2),
    type="l", lty=1, ylab="Density", xlab=expression(theta), main="")
    legend(x="topleft", legend=c(expression(pi(theta)),
    expression(f(y~"|"~theta)), expression(p(theta~"|"~y))), lty=1:3)

# likelihood with more observations
y <- c(6, 5, 6, 4, 7)
n <- length(y)
sigma <- 1
points(x=x, y=dnorm(x=x, mean=mean(y), sd=sigma/n), type="l", lty=2)

# posterior with more observations
postMean <- ( (sigma^2/n)*mu + tau^2*mean(y) ) / ( (sigma^2/n)*mu + tau^2 )
postVar <- ( (sigma^2/n)*tau^2 ) / ( (sigma^2/n) + tau^2 )
points(x=x, y=dnorm(x=x, mean=postMean, sd=sqrt(postVar)), type="l", lty=3)

# MAP with more observations
map <-  x[which.max(dnorm(x=x, mean=postMean, sd=sqrt(postVar)))]
cat("MAP:", map, "(", map-3*sqrt(postVar),",",map+3*sqrt(postVar),")\n")

### Monte Carlo sampling

<img src="Images/MonteCarlo.jpeg" width="600" height="600" />

To derive the posterior distribution we can also draw random samples from it instead of directly calculating its parameters.

This procedure is often called _Monte Carlo sampling_ after the city famous for its casinos.

In the previous example of the Normal/Normal model with multiple observations, we were able to calculate the posterior mean ($4.43$) and posterior variance ($0.17$).
From these parameters, we were able to derive (and plot) the density function, the posterior probability itself.
Alternatively, we can randomly sample directly from the posterior distribution.

In [ ]:
## Monte Carlo sampling
par(mfrow=c(3,1))

# posterior
x <- seq(2,8,0.01)
postMean <- 4.43
postVar <- 0.16
plot(x=x, y=dnorm(x=x, mean=postMean, sd=sqrt(postVar)), type="l", lty=1,
    ylab="Density", xlab=expression(theta), main=expression(p(theta~"|"~y)),
    ylim=c(0,1.2), xlim=c(2,8))

# sampling
y_sampled_1 <- rnorm(n=50, mean=postMean, sd=sqrt(postVar))
hist(y_sampled_1, breaks=20, freq=F, lty=2, col="grey", ylim=c(0,1.2), xlim=c(2,8),
    sub="50 samples", main=expression(p(theta~"|"~y)), xlab=expression(theta))
    points(x=x, y=dnorm(x=x, mean=postMean, sd=sqrt(postVar)), type="l", lty=1)

# more sampling
y_sampled_2 <- rnorm(n=1e6, mean=postMean, sd=sqrt(postVar))
hist(y_sampled_2, breaks=20, freq=F, lty=2, col="grey", ylim=c(0,1.2), xlim=c(2,8),
    sub="1e6 samples", main=expression(p(theta~"|"~y)), xlab=expression(theta))
points(x=x, y=dnorm(x=x, mean=postMean, sd=sqrt(postVar)), type="l", lty=1, ylab="Density",
    xlab=expression(theta), main=expression(p(theta~"|"~y)), sub="1e6 samples")

The more sampling we do, the closer our sampled distribution will be to the "true" posterior distribution. With $50$ samples the empirical posterior mean is $4.444$ while with $1e6$ samples we have an empirical posterior mean of $4.429$ which is very close to our direct estimate of $4.43$. With $50$ samples the empirical posterior variance is $0.105$ while with $1e6$ samples we have an empirical posterior variance of $0.159$ which is very close to our direct estimate of 0.16.

In this simple Normal/Normal case, Monte Carlo methods are not strictly necessary since the integral in the denominator of Bayes' theorem can be evaluated in closed form.
In these cases, it is preferable to derive a smooth curve rather an a histogram of sampled values, and have the corresponding exact values for the posterior parameters.

However, there are cases where, given the choice for the likelihood and prior functions, this integral cannot be evaluated.
In these conditions, Monte Carlo methods are to be preferred for estimating, or rather approximating, the posterior distribution.

As any sample can be drawn from any posterior regardless of the number of unknown parameters $\theta$, we have the ability to work on problems with (theoretically) unlimited complexity, at the price of not obtaining an exact form for the posterior and performing a large number of samplings.

### Intended Learning Outcomes

At the end of this part you are now be able to:
* appreciate the use of Bayesian statistics in life sciences,
* formulate and explain Bayes’ theorem,
* describe a Normal-Normal model and implement it in R with or without Monte Carlo sampling,